In [1]:
import os
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load Spotify credentials
credentials_path = r'../creds/spotify_credentials.json'

with open(credentials_path, 'r') as file:
    creds = json.load(file)

# Initialize Spotify client
auth_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(auth_manager=auth_manager)

In [2]:
def initialize_spotify_client():
    # Set the necessary Spotify API scopes
    scope = "user-library-read playlist-read-private"

    # Check if the environment variables are set
    client_id = os.getenv('SPOTIPY_CLIENT_ID')
    client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
    redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

    if not all([client_id, client_secret, redirect_uri]):
        raise ValueError("Please set the SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, and SPOTIPY_REDIRECT_URI environment variables.")

    # Initialize the Spotify OAuth client
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                                  client_secret=client_secret,
                                                  redirect_uri=redirect_uri,
                                                  scope=scope))

    return sp
sp = initialize_spotify_client()

ValueError: Please set the SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, and SPOTIPY_REDIRECT_URI environment variables.

In [ ]:
# Use if loading spotify credentials from global variables
# client_id = os.getenv('SPOTIPY_CLIENT_ID')
# client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')


# auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
def extract_playlist(sp, playlist_url):
    """Retrieve tracks, audio features, and metadata from a Spotify playlist URL."""
    # Extract playlist ID from URL
    playlist_id = playlist_url.split('/')[-1].split('?')[0]

    # Get playlist metadata and tracks
    playlist_tracks = sp.playlist_tracks(playlist_id)['items']

    tracks_dict = {}
    for track in playlist_tracks:
        track_info = track['track']
        track_id = track_info['id']

        # Get audio features
        audio_features = sp.audio_features(track_id)[0]
        audio_features = {k: v for k, v in audio_features.items()
                          if k not in ['type', 'analysis_url', 'duration_ms']}

        # Get artist information
        artists = [{'artist_name': artist['name'],
                    'artist_genres': sp.artist(artist['id'])['genres']}
                   for artist in track_info['artists']]

        # Add track information to the dictionary
        tracks_dict[track_id] = {
            'track_name': track_info['name'],
            'artists': artists,
            'audio_features': audio_features
        }

    return tracks_dict


def spotify_to_camelot(key, mode):
    camelot_map = {
        (0, 0): '5A', (0, 1): '8B', (1, 0): '12A', (1, 1): '3B',
        (2, 0): '7A', (2, 1): '10B', (3, 0): '2A', (3, 1): '3B',
        (4, 0): '9A', (4, 1): '12B', (5, 0): '4A', (5, 1): '7B',
        (6, 0): '11A', (6, 1): '2B', (7, 0): '6A', (7, 1): '9B',
        (8, 0): '1A', (8, 1): '4B', (9, 0): '8A', (9, 1): '11B',
        (10, 0): '3A', (10, 1): '6B', (11, 0): '10A', (11, 1): '1B'
    }
    return camelot_map.get((key, mode), None)


def add_camelot_key_to_playlist(tracks_dict):
    for track_id, track_info in tracks_dict.items():
        key = track_info['audio_features']['key']
        mode = track_info['audio_features']['mode']
        camelot_key = spotify_to_camelot(key, mode)
        track_info['audio_features']['camelot_key'] = camelot_key
    return tracks_dict


def select_first_song(song_database, first_song=None):
    setlist = []
    remaining_tracks = list(song_database.values())  # Convert dict_values to list

    if first_song is None:
        first_song = random.choice(remaining_tracks)
        remaining_tracks.remove(first_song)  # Remove the chosen song from the playlist
    else:
        for i, track in enumerate(remaining_tracks):
            if track['track_name'] == first_song:
                first_song = remaining_tracks.pop(i)  # Remove the chosen song from the playlist
                break
        else:
            raise ValueError(f"Track '{first_song}' not found in the playlist.")
        
    setlist.append(first_song)
    
    return setlist, remaining_tracks


def get_recommendations(setlist, music_pool, bpm_range=0.05):
    current_song = setlist[-1]
    current_bpm = current_song['audio_features']['tempo']
    current_key = current_song['audio_features']['camelot_key']

    bpm_min, bpm_max = current_bpm * (1 - bpm_range), current_bpm * (1 + bpm_range)
    filtered_songs = [song for song in music_pool
                      if bpm_min <= song['audio_features']['tempo'] <= bpm_max]

    for song in filtered_songs:
        song_key = song['audio_features']['camelot_key']
        song['key_compatibility_score'] = calculate_key_compatibility(current_key, song_key)

    sorted_songs = sorted(filtered_songs, key=lambda x: x['key_compatibility_score'], reverse=True)

    top_candidates = [sorted_songs[0]]
    for candidate in sorted_songs[1:]:
        if candidate['key_compatibility_score'] == top_candidates[0]['key_compatibility_score']:
            audio_sim, genre_sim = calculate_similarity_scores(top_candidates[0], candidate)
            similarity_score = 0.9 * audio_sim + 0.1 * genre_sim

            if similarity_score > top_candidates[0]['key_compatibility_score']:
                top_candidates = [candidate]
            elif similarity_score == top_candidates[0]['key_compatibility_score']:
                top_candidates.append(candidate)
        else:
            break

    return top_candidates


def calculate_similarity_scores(song1, song2):
    song1_features = [song1['audio_features'][feat] for feat in
                      ['speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
    song2_features = [song2['audio_features'][feat] for feat in
                      ['speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

    audio_similarity = cosine_similarity([song1_features], [song2_features])[0][0]

    song1_genres = set(genre for artist in song1['artists'] for genre in artist['artist_genres'])
    song2_genres = set(genre for artist in song2['artists'] for genre in artist['artist_genres'])

    genre_similarity = len(song1_genres.intersection(song2_genres)) / len(song1_genres.union(song2_genres))

    return audio_similarity, genre_similarity


def calculate_key_compatibility(key1, key2):
    # Same key
    if key1 == key2:
        return 1.0

    # Relative key
    if key1[-1] != key2[-1] and key1[:-1] == key2[:-1]:
        return 0.9

    # Perfect fifth up or down
    key1_num, key2_num = int(key1[:-1]), int(key2[:-1])
    if (key1_num - key2_num) % 12 == 7 and key1[-1] == key2[-1]:
        return 0.8
    if (key2_num - key1_num) % 12 == 7 and key1[-1] == key2[-1]:
        return 0.8

    # Perfect fifth up or down to the relative key
    if (key1_num - key2_num) % 12 == 7 and key1[-1] != key2[-1]:
        return 0.7
    if (key2_num - key1_num) % 12 == 7 and key1[-1] != key2[-1]:
        return 0.7

    # Perfect fourth up or down to the relative key
    if (key1_num - key2_num) % 12 == 5 and key1[-1] != key2[-1]:
        return 0.7
    if (key2_num - key1_num) % 12 == 5 and key1[-1] != key2[-1]:
        return 0.7

    # Parallel key modulation
    if key1[:-1] == key2[:-1] and key1[-1] != key2[-1]:
        return 0.6

    return 0


def select_next_song(setlist, music_pool, bpm_range=0.05, max_songs=30):
    while music_pool and len(setlist) < max_songs:
        top_candidates = get_recommendations(setlist, music_pool, bpm_range)
        
        if top_candidates:
            if len(top_candidates) == 1:
                next_song = top_candidates[0]
            else:
                # If multiple candidates have the same top score, choose randomly
                next_song = random.choice(top_candidates)
            
            # Remove the selected song from the music pool
            music_pool.remove(next_song)
            
            # Add the selected song to the setlist
            setlist.append(next_song)
        else:
            # If no songs within the current BPM range, increase the range
            bpm_range = 0.08
    
    # Print the number of songs in the final setlist
    print(f"The final setlist contains {len(setlist)} songs.")
    
    # Print the final setlist in the requested format
    for song in setlist:
        artists = ', '.join([artist['artist_name'] for artist in song['artists']])
        track_name = song['track_name']
        bpm = song['audio_features']['tempo']
        camelot_key = song['audio_features']['camelot_key']
        print(f"[{artists}] - [{track_name}] ({bpm:.2f} BPM, {camelot_key})")
    
    return setlist, music_pool

# Get playlist tracks and features
playlist_url = 'https://open.spotify.com/playlist/3B0Xfhq3aaUZk75wHAZrCs?si=eb8b4aaa1ec04ce3'
music_bank = extract_playlist(sp, playlist_url)
music_bank = add_camelot_key_to_playlist(music_bank)
setlist, music_pool = select_first_song(music_bank, randomize=True)
setlist, music_pool = select_next_song(setlist, music_pool)


The final setlist contains 30 songs.
[Party Pupils] - [Break It Down] (127.02 BPM, 9B)
[Ark Patrol, Phantoms, Veronika Redd] - [Let Go - Phantoms Remix] (124.00 BPM, 9B)
[Bleu Clair] - [Sand Dunes] (126.00 BPM, 9B)
[Bleu Clair] - [Funk Accelerator] (126.02 BPM, 9B)
[Young & Sick] - [Queen of the Valley] (119.95 BPM, 9B)
[The Floozies, The Sponges] - [Hot Guy Alert] (117.98 BPM, 9B)
[keshi] - [us] (118.32 BPM, 4B)
[The Knocks, Mallrat] - [R U HIGH (feat. Mallrat)] (120.94 BPM, 4B)
[Kaskade] - [Hot Wheels] (118.00 BPM, 4A)
[Darius, Devin Tracy] - [EASE YOUR MIND] (113.98 BPM, 4A)
[JNTHN STEIN] - [Jam] (114.96 BPM, 4A)
[Fred again..] - [Kyle (i found you)] (112.02 BPM, 4A)
[Doja Cat, SZA] - [Kiss Me More (feat. SZA)] (110.97 BPM, 4B)
[Franc Moody] - [I'm in a Funk] (109.97 BPM, 9B)
[The Marías] - [Ruthless] (105.00 BPM, 11B)
[The Knocks, Studio Killers] - [Bedroom Eyes (feat. Studio Killers)] (109.02 BPM, 11B)
[Jo Hill] - [HONEYMOON] (110.01 BPM, 10B)
[Darius, Lo Village] - [MATERIAL GIRL